In [ ]:
import pandas as pd
import numpy as np

In [ ]:
movies = pd.read_csv('data/tmdb_5000_movies.csv')
credits = pd.read_csv('data/tmdb_5000_credits.csv') 

In [ ]:
movies.head()

In [ ]:
movies=movies.merge(credits,on='title')

In [ ]:
movies.head()

In [ ]:
# genres
#id
#keywords
#title
#overview
#cast
#crew
movies=movies[['movie_id','title','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()


In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
import ast
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres'].apply(convert)


In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
movies.head()

In [ ]:
import ast
def convertcast(obj):
    L=[]
    count=0;
    for i in ast.literal_eval(obj):
        if(count!=3):
            L.append(i['name'])
            count+=1;    
        else:
            break  
    return L

In [ ]:
movies['cast'].apply(convertcast)

In [ ]:
movies['cast']=movies['cast'].apply(convertcast)

In [ ]:
movies.head()

In [ ]:
import ast
def convertdirec(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
            
        
    return L

In [ ]:
movies['crew']=movies['crew'].apply(convertdirec)

In [ ]:
movies.head()

In [ ]:
import ast
def convertover(obj):
    L=[]
    temp=""
    for i in (obj):
        if(i!=' '):
            temp+=i
        else:
            L.append(temp)
            temp=""
    return L

In [ ]:
movies.head()

In [ ]:
movies=movies[['movie_id','title','genres','keywords','cast','crew']]

In [ ]:
movies.head()

In [ ]:
movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])


In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['genres']+movies['keywords']+movies['cast']+movies['crew']


In [ ]:
new_df=movies[['movie_id','title','tags']]

In [ ]:
new_df

In [ ]:
new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names()

In [ ]:
import nltk

In [ ]:
pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
movies.head()

In [ ]:
movies['tags']

In [ ]:
movies['tags']

In [ ]:
new_df['tags'].apply(stem)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity[0]

In [ ]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)
    

In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [61]:
pickle.dump(similarity,open('similarity.pkl','wb'))